In [1]:
import pandas as pd
import numpy as np
from talib import abstract

In [2]:
df = pd.read_csv('raw_data.csv')
df.set_index(['Ticker', 'Date'], inplace=True)
df = df.dropna(subset=['Volume'])
del df['Unnamed: 7']  # Why is this column appearing?
df = df.sort_index()

In [3]:
tickers = df.index.levels[0].unique()

In [4]:
# Simple Moving Average
SMA = abstract.SMA
ROCP = abstract.ROCP
MACD = abstract.MACD
RSI = abstract.RSI
BBANDS = abstract.BBANDS

In [5]:
cols = ["5MA_Close", "5MA_Close_Returns", "10MA_Close", "10MA_Close_Returns", "20MA_Close", "20MA_Close_Returns", "30MA_Close", "30MA_Close_Returns", "ROCP_Close", "ROCP_Open", "ROCP_High", "ROCP_Low",  "ROCP_Volume", "DIF", "DEA", "MACD", "DIFROCP", "DEAROCP", "MACDROCP", "RSI6", "RSI16", "RSI24", "RSI6ROCP", "RSI16ROCP", "RSI24ROCP", "BBAND_Up", "BBAND_Mid", "BBAND_Low", "ROCP_Price_Volume", "5MA_Volume", "5MA_Volume_Returns", "10MA_Volume", "10MA_Volume_Returns", "20MA_Volume", "20MA_Volume_Returns", "30MA_Volume","30MA_Volume_Returns",]
df_h = pd.DataFrame(np.nan, index=df.index, columns=cols)

In [6]:
for t in tickers:
    # @TODO: Remove 828 from data set.
    if t == 828:
        continue
        
    df_t = df.loc[t]
    df_t = pd.concat([df_t], keys=[t], names=['Ticker'])

    inputs = {
    'open': df_t['Open'],
    'high': df_t['High'],
    'low': df_t['Low'],
    'close': df_t['Close'],
    'volume': df_t['Volume']
    }
    
    df_h.loc[t]["5MA_Close"] = SMA(inputs, timeperiod=5, price='close')
    df_h.loc[t]["5MA_Close_Returns"] = df_h.loc[t]["5MA_Close"].pct_change()  # Isn't this just this ROC?
    df_h.loc[t]["10MA_Close"] = SMA(inputs, timeperiod=10, price='close')
    df_h.loc[t]["10MA_Close_Returns"] = df_h.loc[t]["10MA_Close"].pct_change()  # Isn't this just this ROC?
    df_h.loc[t]["20MA_Close"] = SMA(inputs, timeperiod=20, price='close')
    df_h.loc[t]["20MA_Close_Returns"] = df_h.loc[t]["20MA_Close"].pct_change()  # Isn't this just this ROC?
    df_h.loc[t]["30MA_Close"] = SMA(inputs, timeperiod=30, price='close')
    df_h.loc[t]["30MA_Close_Returns"] = df_h.loc[t]["30MA_Close"].pct_change()  # Isn't this just this ROC? 

    df_h.loc[t]["ROCP_Close"] = ROCP(inputs, timeperiod=1, price='close')
    df_h.loc[t]["ROCP_Open"] = ROCP(inputs, timeperiod=1, price='open')
    df_h.loc[t]["ROCP_High"] = ROCP(inputs, timeperiod=1, price='high')    
    df_h.loc[t]["ROCP_Low"] = ROCP(inputs, timeperiod=1, price='low')
    df_h.loc[t]["ROCP_Volume"] = ROCP(inputs, timeperiod=1, price='volume')
    
    df_h.loc[t]["DIF"], df_h.loc[t]["DEA"], df_h.loc[t]["MACD"] = MACD(inputs, fastperiod=12, slowperiod=26, signalperiod=9)
    df_h.loc[t]["DIFROCP"] = ROCP({'close': df_h.loc[t]["DIF"]}, timeperiod=1, price='close')
    df_h.loc[t]["DEAROCP"] = ROCP({'close': df_h.loc[t]["DEA"]}, timeperiod=1, price='close')
    df_h.loc[t]["MACDROCP"] = ROCP({'close': df_h.loc[t]["MACD"]}, timeperiod=1, price='close')
    
    df_h.loc[t]["RSI6"] = RSI(inputs, timeperiod=6, price='close')    
    df_h.loc[t]["RSI16"] = RSI(inputs, timeperiod=16, price='close')    
    df_h.loc[t]["RSI24"] = RSI(inputs, timeperiod=24, price='close')    
    df_h.loc[t]["RSI6ROCP"] = ROCP({'close': df_h.loc[t]["RSI6"]}, timeperiod=1, price='close')
    df_h.loc[t]["RSI16ROCP"] = ROCP({'close': df_h.loc[t]["RSI16"]}, timeperiod=1, price='close')
    df_h.loc[t]["RSI24ROCP"] = ROCP({'close': df_h.loc[t]["RSI24"]}, timeperiod=1, price='close')
    df_h.loc[t]["BBAND_Up"], df_h.loc[t]["BBAND_Mid"], df_h.loc[t]["BBAND_Low"] = BBANDS(inputs, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0, price='close')
    
    df_h.loc[t]["ROCP_Price_Volume"] = df_h.loc[t]["ROCP_Volume"] * df_h.loc[t]["ROCP_Close"]
    
    df_h.loc[t]["5MA_Volume"] = SMA(inputs, timeperiod=5, price='volume')
    df_h.loc[t]["5MA_Volume_Returns"] = df_h.loc[t]["5MA_Close"].pct_change()  # Isn't this just this ROC?
    df_h.loc[t]["10MA_Volume"] = SMA(inputs, timeperiod=10, price='volume')
    df_h.loc[t]["10MA_Volume_Returns"] = df_h.loc[t]["10MA_Volume"].pct_change()  # Isn't this just this ROC?
    df_h.loc[t]["20MA_Volume"] = SMA(inputs, timeperiod=20, price='volume')
    df_h.loc[t]["20MA_Volume_Returns"] = df_h.loc[t]["20MA_Volume"].pct_change()  # Isn't this just this ROC?
    df_h.loc[t]["30MA_Volume"] = SMA(inputs, timeperiod=30, price='volume')
    df_h.loc[t]["30MA_Volume_Returns"] = df_h.loc[t]["30MA_Volume"].pct_change()  # Isn't this just this ROC? 

In [7]:
def merge_multi(self, df, on):
        return self.reset_index().join(df,on=on).set_index(self.index.names)
    
pd.DataFrame.merge_multi = merge_multi
df = df.merge_multi(df_h, on=['Ticker', 'Date'])

In [ ]:
df.to_csv('processed_data')